Imports

In [2]:
!pip install nnsight
!pip install sae_lens

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.1/102.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [47]:
# Import necessary libraries
from nnsight import LanguageModel
import torch
import pandas as pd

# Initialize the language model with specific parameters
model = LanguageModel("google/gemma-2b-it", trust_remote_code=True, device_map="cuda:0", low_cpu_mem_usage=True, torch_dtype=torch.float16)
model.requires_grad_(False)



from sae_lens import SAE
layer = 6

# get the SAE for this layer
sae, cfg_dict, _ = SAE.from_pretrained(
    release = "gemma-2b-res-jb",
    sae_id = f"blocks.{layer}.hook_resid_post",
    device = 'cuda:0'
)

Parameters

## Main code prompt


In [48]:
v = ["How does does financial sentiment affect credit rating, particular related to stock price?", "what is IBM credit rating and how does  it effect lending?"]

In [49]:
#@title Self-explanation in 18 lines
feature = 471  #@param {type: "integer"}
#feature = 5892  #@param {type: "integer"}
# feature = 9415
scale = 76  #@param {type: "number"}
se_demo = True  #@param {type: "boolean"}
max_new_tokens = 80  #@param {type: "integer"}
n_generate = 1  #@param {type: "integer"}
vals = ['positive', 'neutral', 'negative']
samples = pd.read_excel('https://raw.githubusercontent.com/ashater/sae/master/sentence_sentiment_gpt4o.xlsx')

In [51]:
full_res = []
#v = range(0,10)
for ismpl in v:


    prompt = """<start_of_turn>user\nCan you rate  the following sentnece as a financial credit sentiment, very positive,
    somewhat positive, neutral, somewhat negative, very negative? """ + samples.iloc[ismpl].Sentence + '<end_of_turn>\n<start_of_turn>model\nThe sentiment of the setence is "X"'

    #prompt = '<start_of_turn>user\nWhat is the meaning of the word "X"?<end_of_turn>\n<start_of_turn>model\nThe meaning of the word "X" is "'

    # prompt = '<start_of_turn>user\n' + ismpl +  '<end_of_turn>\n<start_of_turn>model X'
    positions = [i for i, a in enumerate(model.tokenizer.encode(prompt)) if model.tokenizer.decode([a]) == "X"]
    with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
        # vector steering section
        vector = sae.W_dec[[feature]]
        vector = vector / vector.norm()
        vector = vector * scale
        # setting vector into layer 2
        for position in positions:
          model.model.layers[2].output[0][:, position] = vector

        # get final output saved
        out = model.generator.output.save()
        # capture at layer 6 or 12
        resid = model.model.layers[12].output.save()

    features = sae.encode(resid[0])
    #print(features)
    summed_activations = features.abs().sum(dim=1) # Sort by max activations

    top_activations_indices = summed_activations.topk(20).indices # Get indices of top 20
    print("Top actiivation index", top_activations_indices)
    for i, l in enumerate(model.tokenizer.batch_decode(out)):
        s = repr(l.partition(prompt)[2].partition("<eos>")[0])
        print(prompt + "|"+ s + "|" + str(top_activations_indices))
        # print(str(top_activations_indices))


TypeError: Cannot index by location index with a non-integer key

## summed activations across tokens

In [7]:
summed_activations.shape

torch.Size([1, 16384])

## Get features for each token

In [8]:
features.shape

torch.Size([1, 29, 16384])

In [17]:
# Step 1: Load the necessary libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import umap
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline

# Load your Excel sheet containing feature numbers and sentences
# Update the file path if needed
file_path = 'https://raw.githubusercontent.com/ashater/sae/master/explains_sae.xlsx'  # Replace with your file path
df = pd.read_excel(file_path)
df.columns = ['Feature_Number', 'Sentence']
df.head()

,Feature_Number,Sentence
0,0,"['\'a place or area of something or someone"". ..."
1,1,['\'to arrive or be at a particular place or t...
2,2,"['\'going back"" or ""in the past"". It is often ..."
3,3,"['\'to take or receive with the mouth"". It can..."
4,4,"['\'the center or core of something"". It is of..."


In [18]:
!pip install transformers  # Install the transformers library (version 4.31.0) for text summarization

In [20]:
from transformers import pipeline

summarizer = pipeline("summarization")  # Load a pre-trained summarization model

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


In [23]:
sentence = df['Sentence'][0]
sentence

'[\'\\\'a place or area of something or someone". It can refer to a physical place, such as a room or a box, or to a more abstract place, such as a feeling or a thought.\\\'\', \'\\\'a place where something is received or taken". It often refers to a place where something is delivered or dropped off. \\\\n\\\\nIn technical terms, a receiver is a device used to receive a signal or\\\'\', \'\\\'a place where something or someone is received or taken." It can also mean "a place to receive or store something."\\\'\', \'\\\'a place that receives or takes something". It can refer to:\\\\n\\\\n- A physical place, such as a mailbox, a library, or a store.\\\\n- The area of a network or computer\\\'\', \'\\\'a place or object where something is taken or received". It can refer to:\\\\n\\\\n* A physical place, like a mailbox or a storage room.\\\\n* A digital device, like a camera or\\\'\', \'\\\'a place where something is received". It can refer to a physical place, such as a house or a store, 

In [37]:
summary_text = summarizer(sentence, max_length=30, min_length=10, do_sample=False)[0]['summary_text']

In [38]:
!pip install gensim  # Install gensim for LDA

import nltk
nltk.download('stopwords')  # Download stopwords for preprocessing
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [39]:
# Define your sentences
sentences = [
    "This is the first sentence about topic A.",
    "Another sentence related to topic A.",
    "This sentence introduces topic B.",
    "More details about topic B.",
    "A sentence about topic C."
]


In [40]:
# Preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalnum() and token not in stop_words]
    return tokens

processed_sentences = [preprocess_text(sentence) for sentence in sentences]

# Create dictionary and corpus
dictionary = corpora.Dictionary(processed_sentences)
corpus = [dictionary.doc2bow(text) for text in processed_sentences]

# Train LDA model
lda_model = models.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=15)

# Get dominant topic for each sentence
dominant_topics = [max(lda_model[bow], key=lambda item: item[1])[0] for bow in corpus]

# Print dominant topics and corresponding sentences
for i, topic in enumerate(dominant_topics):
    print(f"Sentence: {sentences[i]}")
    print(f"Dominant Topic: {topic}")
    print("-" * 20)

# Print overall topic distribution
topic_distribution = lda_model.print_topics()
for topic in topic_distribution:
    print(topic)

Sentence: This is the first sentence about topic A.
Dominant Topic: 1
--------------------
Sentence: Another sentence related to topic A.
Dominant Topic: 1
--------------------
Sentence: This sentence introduces topic B.
Dominant Topic: 2
--------------------
Sentence: More details about topic B.
Dominant Topic: 2
--------------------
Sentence: A sentence about topic C.
Dominant Topic: 0
--------------------
(0, '0.224*"sentence" + 0.221*"c" + 0.221*"topic" + 0.056*"first" + 0.056*"detail" + 0.056*"introduces" + 0.056*"b" + 0.056*"related" + 0.056*"another"')
(1, '0.236*"sentence" + 0.233*"topic" + 0.133*"related" + 0.133*"another" + 0.133*"first" + 0.033*"introduces" + 0.033*"b" + 0.033*"detail" + 0.033*"c"')
(2, '0.234*"b" + 0.234*"topic" + 0.134*"detail" + 0.134*"introduces" + 0.130*"sentence" + 0.034*"first" + 0.034*"another" + 0.034*"related" + 0.034*"c"')


In [35]:
summary_text

' It can refer to a physical place, such as a room or a box, or to a more abstract place . It often refers to'

In [32]:
import nltk
nltk.download('punkt')  # Download necessary data for tokenization
nltk.download('punkt_tab')

from nltk.tokenize import sent_tokenize

phrases = sent_tokenize(summary_text)  # Split the summary into phrases (sentences)

# Limit to 5 phrases if necessary
phrases = phrases[:5]

print(phrases)

[' It can refer to a physical place, such as a room or a box, or to a more abstract place, like a feeling or a thought .', 'It often refers to a place where something is delivered or dropped off .']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [33]:
print(phrases)

[' It can refer to a physical place, such as a room or a box, or to a more abstract place, like a feeling or a thought .', 'It often refers to a place where something is delivered or dropped off .']


In [ ]:

# Assuming your dataframe has 'Feature_Number' and 'Sentence' columns
# Update column names accordingly if they differ
df.head()

# Step 2: Summarize the sentences (manual summarization example)
# You can use NLP techniques such as Extractive Summarization or custom heuristics
# For simplicity, let's assume we manually created a "Summary" column
# If you're automating this, you can use NLP libraries like Hugging Face transformers or other summarizers

# Adding a 'Summary' column (replace with actual logic)
df['Summary'] = df['Sentence'].apply(lambda x: ' '.join(x.split()[:5]))  # Example summarization




In [ ]:
df.head()

In [ ]:
# Step 3: Filter financial-related phrases (this is an example filter, you can enhance it)
# You can expand this list with domain-specific terms
financial_keywords = ['finance', 'stock', 'market', 'investment', 'credit', 'risk', 'banking', 'economy']

# Filter phrases that contain financial keywords
df['Is_Financial'] = df['Summary'].apply(lambda x: any(keyword in x.lower() for keyword in financial_keywords))

# Step 4: Vectorizing the text using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df[df['Is_Financial']]['Summary'])  # Only use financial summaries

# Step 5: Apply UMAP for dimensionality reduction and visualization
umap_model = umap.UMAP(n_neighbors=10, min_dist=0.3, metric='cosine')
X_umap = umap_model.fit_transform(X.toarray())

# Step 6: Plot the UMAP result
plt.figure(figsize=(10, 8))
plt.scatter(X_umap[:, 0], X_umap[:, 1], c='blue', edgecolor='k', s=50)
plt.title('UMAP Projection of Financial Topics')
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.show()

# Optionally: Save the UMAP results for further analysis
umap_df = pd.DataFrame(X_umap, columns=['UMAP1', 'UMAP2'])
umap_df['Feature_Number'] = df[df['Is_Financial']]['Feature_Number'].values
umap_df.to_csv('/content/umap_results.csv', index=False)

# Display the UMAP results
umap_df.head()